In [2]:
import gensim

assert gensim.models.doc2vec.FAST_VERSION > -1

In [3]:
import gensim
import gensim.test.utils

# Set file names for train and test data
lee_train_file = gensim.test.utils.datapath('lee_background.cor')
lee_test_file = gensim.test.utils.datapath('lee.cor')

In [4]:
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

print(train_corpus[2])
print(test_corpus[2])

TaggedDocument<['the', 'national', 'road', 'toll', 'for', 'the', 'christmas', 'new', 'year', 'holiday', 'period', 'stands', 'at', 'eight', 'fewer', 'than', 'for', 'the', 'same', 'time', 'last', 'year', 'people', 'have', 'died', 'on', 'new', 'south', 'wales', 'roads', 'with', 'eight', 'fatalities', 'in', 'both', 'queensland', 'and', 'victoria', 'western', 'australia', 'the', 'northern', 'territory', 'and', 'south', 'australia', 'have', 'each', 'recorded', 'three', 'deaths', 'while', 'the', 'act', 'and', 'tasmania', 'remain', 'fatality', 'free'], [2]>
['the', 'united', 'states', 'government', 'has', 'said', 'it', 'wants', 'to', 'see', 'president', 'robert', 'mugabe', 'removed', 'from', 'power', 'and', 'that', 'it', 'is', 'working', 'with', 'the', 'zimbabwean', 'opposition', 'to', 'bring', 'about', 'change', 'of', 'administration', 'as', 'scores', 'of', 'white', 'farmers', 'went', 'into', 'hiding', 'to', 'escape', 'round', 'up', 'by', 'zimbabwean', 'police', 'senior', 'bush', 'administrat

In [5]:
import gensim.models

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [6]:
model.build_vocab(train_corpus)

In [7]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [8]:
model.dv

In [9]:
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

[-0.15804976 -0.2499155  -0.11658205  0.21830133  0.01257771 -0.05956412
  0.11286413 -0.03740422 -0.1275653  -0.04258005  0.16517814  0.00358914
 -0.03623567 -0.06548166 -0.19615334 -0.2485502   0.18084258  0.18186264
  0.2080098  -0.08744554 -0.08761725  0.02893791  0.21769735 -0.09310921
 -0.01198547 -0.14353824 -0.33309653 -0.01825209 -0.18215586 -0.08988088
  0.3794694  -0.04330229  0.24006465  0.15158856  0.17017049  0.14419761
  0.00641705 -0.17949751 -0.08991947 -0.05096073 -0.01911813 -0.00208266
 -0.12335294 -0.04781395  0.20238739  0.02927922 -0.12975693 -0.00910016
  0.14824723  0.00612068]


In [10]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 292, 1: 8})


In [11]:
import random

# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (22): «the biowarfare expert under scrutiny in the anthrax attacks declared am not the anthrax killer and lashed out today against attorney general john ashcroft for calling him person of interest in the investigation for the second time in two weeks the scientist went before throng of reporters outside his lawyer office to profess his innocence and decry the attention from law enforcers that he contends has destroyed his life»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec<dm/m,d50,n5,w5,mc2,s0.001,t3>:

MOST (23, 0.6996737122535706): «americans fears about airplane security continue to increase after man made it through two separate flights with loaded gun in his carry on luggage the man was finally stopped before boarding third plane in memphis the man had travelled from florida to atlanta and then atlanta to memphis he was attempting to board his return flight last night when he was stopped by security personnel for random check they discovered loaded mm beretta semi auto

In [12]:
pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install gensim pandas datasets pyarrow nltk

Note: you may need to restart the kernel to use updated packages.


завдання 0

In [19]:
pip install fsspec

Note: you may need to restart the kernel to use updated packages.


In [1]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from datasets import load_dataset # Для завантаження наборів даних з Hugging Face Hub
import nltk # Для токенізації
import multiprocessing # Для використання багатоядерності

print(f"Версія Gensim: {gensim.__version__}")
# Переконуємося, що доступна швидка C-версія Doc2Vec
assert gensim.models.doc2vec.FAST_VERSION > -1
print("Перевірка Gensim FAST_VERSION пройдена.")

nltk.download('punkt', quiet=True) # quiet=True, щоб зменшити кількість виводу, якщо все вже є

dataset_train = load_dataset("stanfordnlp/imdb", split="train[:5000]") # Перші 5000 зразків для тренування
dataset_test = load_dataset("stanfordnlp/imdb", split="test[:1000]")   # Перші 1000 зразків для тестування
print(f"Завантажено {len(dataset_train)} тренувальних документів та {len(dataset_test)} тестових документів.")

train_corpus = []
for i, item in enumerate(dataset_train):
    text_document = item['text']
    tokens = nltk.word_tokenize(text_document.lower())
    train_corpus.append(TaggedDocument(tokens, [i]))

if not train_corpus: # перевірка чи корпус не порожній
    print("Тренувальний корпус порожній. Перевірте завантаження набору даних та попередню обробку.")
    pass # Дозволяємо програмі продовжити; помилка виникне пізніше, якщо корпус порожній.

if train_corpus: # Виводимо приклад, тільки якщо корпус не порожній
    print(f"Перший попередньо оброблений тренувальний документ: {train_corpus[0]}")


# модель Doc2Vec
cores = multiprocessing.cpu_count()
print(f"Ініціалізація моделі Doc2Vec з {cores} робочими потоками...")

model = Doc2Vec(
    vector_size=100,
    min_count=5,
    epochs=20,
    dm=1,
    workers=cores-1 if cores > 1 else 1
)

# Якщо train_corpus порожній, тут виникне помилка.
model.build_vocab(train_corpus)
print(f"Словник побудовано, розмір словника: {len(model.wv.index_to_key)}")

model.train(
    train_corpus,
    total_examples=model.corpus_count,
    epochs=model.epochs
)

model_save_path = "imdb_doc2vec_model_ua.model"
model.save(model_save_path)

#  виведення вектора для нового речення 
new_doc_text = "це був справді дивовижний і чудовий кінематографічний досвід"
new_doc_tokens = nltk.word_tokenize(new_doc_text.lower())

inferred_vector = model.infer_vector(new_doc_tokens)
print(f"\nВиведений вектор для '{new_doc_text}':\n{inferred_vector[:10]}... (перші 10 розмірностей)")

# Перевіряємо, чи існує атрибут 'dv' і чи він не порожній, перш ніж викликати most_similar
if hasattr(model, 'dv') and model.dv:
    sims = model.dv.most_similar([inferred_vector], topn=3)
    print("\nНайбільш схожі документи в тренувальному наборі до нового речення:")
    for doc_tag, similarity in sims:
        original_doc_text = dataset_train[doc_tag]['text'][:200]
        print(f"  Док ID {doc_tag} (Схожість: {similarity:.4f}): \"{original_doc_text}...\"")
else:
    print("\nmodel.dv не заповнений або відсутній. Неможливо знайти схожі документи.")


C:\Users\HP\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Версія Gensim: 4.3.3
Перевірка Gensim FAST_VERSION пройдена.


C:\Users\HP\anaconda3\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\datasets--stanfordnlp--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 322835.45 examples/s]


Завантажено 5000 тренувальних документів та 1000 тестових документів.
Перший попередньо оброблений тренувальний документ: TaggedDocument<['i', 'rented', 'i', 'am', 'curious-yellow', 'from', 'my', 'video', 'store', 'because', 'of', 'all', 'the', 'controversy', 'that', 'surrounded', 'it', 'when', 'it', 'was', 'first', 'released', 'in', '1967.', 'i', 'also', 'heard', 'that', 'at', 'first', 'it', 'was', 'seized', 'by', 'u.s.', 'customs', 'if', 'it', 'ever', 'tried', 'to', 'enter', 'this', 'country', ',', 'therefore', 'being', 'a', 'fan', 'of', 'films', 'considered', '``', 'controversial', "''", 'i', 'really', 'had', 'to', 'see', 'this', 'for', 'myself.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'the', 'plot', 'is', 'centered', 'around', 'a', 'young', 'swedish', 'drama', 'student', 'named', 'lena', 'who', 'wants', 'to', 'learn', 'everything', 'she', 'can', 'about', 'life', '.', 'in', 'particular', 'she', 'wants', 'to', 'focus', 'her', 'attentions', 'to', 'making', 'some', 'sort', 'of', 'do

завдання 1

In [4]:
import gensim
from gensim.models.doc2vec import Doc2Vec
from datasets import load_dataset
import nltk
import random
import os # Додано для перевірки існування файлу

model_path = "imdb_doc2vec_model_ua.model" # Шлях до вашої збереженої моделі

# Перевіряємо, чи існує файл моделі, перш ніж намагатися його завантажити
if not os.path.exists(model_path):
    print(f"Файл моделі '{model_path}' не знайдено.")
    exit() # Завершуємо роботу, якщо моделі немає

model = Doc2Vec.load(model_path)

dataset_for_similarity = load_dataset("stanfordnlp/imdb", split="train[:1000]")
nltk.download('punkt', quiet=True)

# Токенізує текст та переводить його в нижній регістр
def preprocess_text(text):
    return nltk.word_tokenize(text.lower())


#  випадковий документ з  набору даних
random_doc_index = random.randint(0, len(dataset_for_similarity) - 1)
query_document_text = dataset_for_similarity[random_doc_index]['text']
query_document_label = dataset_for_similarity[random_doc_index]['label']

print(f"Запитний документ (ID: {random_doc_index}, Мітка: {'Позитивний' if query_document_label == 1 else 'Негативний'}) ")
print(query_document_text[:500] + "..." if len(query_document_text) > 500 else query_document_text)

query_tokens = preprocess_text(query_document_text)

#отримання вектора для запитного документа за допомогою навченої моделі
inferred_vector = model.infer_vector(query_tokens)

#Пошук найбільш схожих документів у навчальному корпусі моделі
# Перевіряємо, чи існує атрибут 'dv' і чи він не порожній
if hasattr(model, 'dv') and model.dv:
    num_similar = 5
    sims = model.dv.most_similar([inferred_vector], topn=num_similar)

    print(f" {num_similar} найбільш схожих документів з тренувального набору моделі")
    for doc_id_in_model_dv, similarity_score in sims:
        # Перевіряємо, чи індекс знаходиться в межах поточного завантаженого набору
        if doc_id_in_model_dv < len(dataset_for_similarity):
            similar_doc_text = dataset_for_similarity[doc_id_in_model_dv]['text']
            similar_doc_label = dataset_for_similarity[doc_id_in_model_dv]['label']
            label_text = 'Позитивний' if similar_doc_label == 1 else 'Негативний'
            print(f"\nДокумент (ID з model.dv: {doc_id_in_model_dv}, Мітка: {label_text}, Схожість: {similarity_score:.4f}):")
            print(similar_doc_text[:300] + "..." if len(similar_doc_text) > 300 else similar_doc_text)
        else:
            print(f"\nДокумент (ID з model.dv: {doc_id_in_model_dv}, Схожість: {similarity_score:.4f}):")
            print("(Неможливо відобразити текст, оскільки ID виходить за межі поточного завантаженого `dataset_for_similarity`)")
            print("Це означає, що схожий документ був у повному тренувальному наборі моделі, але не в поточній завантаженій підмножині.")
else:
    print("model.dv не знайдено або порожній. Неможливо знайти схожі документи.")


#  перевірка, чи збігаються мітки (якщо доступні)
print(f"\nМітка запитного документа: {'Позитивний' if query_document_label == 1 else 'Негативний'}")
if hasattr(model, 'dv') and model.dv and 'sims' in locals() and sims is not None:
    print("Мітки знайдених схожих документів (з поточного `dataset_for_similarity`):")
    for doc_id_in_model_dv, _ in sims:
        if doc_id_in_model_dv < len(dataset_for_similarity):
            label = dataset_for_similarity[doc_id_in_model_dv]['label']
            print(f"  ID {doc_id_in_model_dv}: {'Позитивний' if label == 1 else 'Негативний'}")

Запитний документ (ID: 511, Мітка: Негативний) 
I don't give much credence to AIDS conspiracy theories but its sociologically interesting to see the phenomenon dramatized. In the early years of the AIDS epidemic, the suffering and paranoia of the scared and dying often generated such dark fantasies. This was especially true in the politically radical and sexually extreme demi-monde of San Francisco. The city, renowned for its beauty, has rarely appeared uglier than in this film. A sense of darkness and decomposition pervades every scene.<br /...
 5 найбільш схожих документів з тренувального набору моделі

Документ (ID з model.dv: 511, Мітка: Негативний, Схожість: 0.9005):
I don't give much credence to AIDS conspiracy theories but its sociologically interesting to see the phenomenon dramatized. In the early years of the AIDS epidemic, the suffering and paranoia of the scared and dying often generated such dark fantasies. This was especially true in the politically rad...

Документ (ID з